In [2]:
import time
import os
import numpy as np
import random

import torch
import torch.nn as nn
from torchtext import data

from args import get_args
from model import SmPlusPlus
from trec_dataset import TrecDataset
from wiki_dataset import WikiDataset
from evaluate import evaluate
from torch.autograd import Variable


In [13]:
# TEXT = data.Field(batch_first=True, tokenize=clean_str_sst)
# LABEL = data.Field(sequential=False)
def set_vectors(field, vector_path):
    if os.path.isfile(vector_path):
        stoi, vectors, dim = torch.load(vector_path)
        field.vocab.vectors = torch.Tensor(len(field.vocab), dim)

        for i, token in enumerate(field.vocab.itos):
            wv_index = stoi.get(token, None)
            if wv_index is not None:
                field.vocab.vectors[i] = vectors[wv_index]
            else:
                # initialize <unk> with U(-0.25, 0.25) vectors
                field.vocab.vectors[i] = torch.FloatTensor(dim).uniform_(-0.25, 0.25)
    else:
        print("Error: Need word embedding pt file")
        print("Error: Need word embedding pt file")
        exit(1)
    return field

QID = data.Field(sequential=False)
AID = data.Field(sequential=False)
QUESTION = data.Field(batch_first=True)
ANSWER = data.Field(batch_first=True)
LABEL = data.Field(sequential=False)
EXTERNAL = data.Field(sequential=False, tensor_type=torch.FloatTensor, batch_first=True, use_vocab=False,
                      preprocessing=data.Pipeline(lambda x: x.split()),
                      postprocessing=data.Pipeline(lambda x, train: [float(y) for y in x]))

train, dev, test = TrecDataset.splits(QID, QUESTION, AID, ANSWER, EXTERNAL, LABEL)

QID.build_vocab(train, dev, test)
AID.build_vocab(train, dev, test)
QUESTION.build_vocab(train, dev, test)
ANSWER.build_vocab(train, dev, test)
LABEL.build_vocab(train, dev, test)
vector_cache = "data/word2vec.trecqa.pt"

QUESTION = set_vectors(QUESTION, vector_cache)
ANSWER = set_vectors(ANSWER, vector_cache)

In [62]:
index2label = np.array(LABEL.vocab.itos)
index2label

array(['<unk>', '0', '1'],
      dtype='<U5')

In [3]:
train.fields

{'aid': <torchtext.data.field.Field at 0x7fd327e15e48>,
 'answer': <torchtext.data.field.Field at 0x7fd327e15ef0>,
 'ext_feat': <torchtext.data.field.Field at 0x7fd327e15f98>,
 'label': <torchtext.data.field.Field at 0x7fd327e15f28>,
 'qid': <torchtext.data.field.Field at 0x7fd327e15dd8>,
 'question': <torchtext.data.field.Field at 0x7fd327e15eb8>}

In [47]:
print(dir(train.examples[1]))


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'aid', 'answer', 'ext_feat', 'fromCSV', 'fromJSON', 'fromTSV', 'fromdict', 'fromlist', 'fromtree', 'label', 'qid', 'question']


In [50]:
type(train.examples[0])

torchtext.data.example.Example

In [4]:
# print(type(train.splits()))
print(dir(train.examples[1]))
print(train[1].question)
train_iter = data.Iterator(train, batch_size=1000, device=-1, train = True, repeat=False,
                                   sort=False, shuffle=True)
train_iter.init_epoch()
for batch_idx, batch in enumerate(train_iter):
    print(dir(batch))
    print(dir(batch.dataset))
    print(batch.fromvars)
    break

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'aid', 'answer', 'ext_feat', 'fromCSV', 'fromJSON', 'fromTSV', 'fromdict', 'fromlist', 'fromtree', 'label', 'qid', 'question']
['who', 'is', 'the', 'author', 'of', 'the', 'book', ',', '``', 'the', 'iron', 'lady', ':', 'a', 'biography', 'of', 'margaret', 'thatcher', "''", '?']
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'aid', 'answer', 'batch_size', 'd

In [5]:
dir(batch)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'aid',
 'answer',
 'batch_size',
 'dataset',
 'ext_feat',
 'fromvars',
 'label',
 'qid',
 'question',
 'train']

In [6]:
batch.label[1].cpu().data.numpy()[0]

1

In [7]:
resume_snapshot = "saves/TREC/multichannel_best_model.pt"
model = torch.load(resume_snapshot, map_location=lambda storage, location: storage)

model.train();
features = model(batch)

/home/w85yang/anaconda3/lib/python3.6/site-packages/torch/serialization.py:294: SourceChangeWarning: source code of class 'model.SmPlusPlus' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [38]:
i = 0
new_train = {"ext_feat":[], "question":[], "answer":[], "label":[]}
new_train["answer"].append(near_answer)
new_train["question"].append(q)
new_train["label"].append(label_dict[l])
new_train["ext_feat"].append(batch.ext_feat[x])
new_batch = data.Batch()
new_batch.batch_size = 1
new_batch.dataset = batch.dataset
setattr(new_batch,"answer",torch.stack(new_train["answer"]))
setattr(new_batch,"question",torch.stack(new_train["question"]))
setattr(new_batch,"ext_feat",torch.stack(new_train["ext_feat"]))
setattr(new_batch,"label",torch.stack(new_train["label"]).view(-1))
# model({question:batch.question[i], answer:batch.answer[i], ext_feat:batch.ext_feat[i]})

NameError: name 'question' is not defined

NameError: name 'new_train' is not defined

In [5]:
features.data.cpu().numpy().shape

(1000, 204)

In [33]:
index2qid = np.array(QID.vocab.itos)
# index2qid


array(['518', '748', '825', '1063', '1090', '134', '1737', '2296', '35',
       '54', '898', '1929', '1074', '1235', '1501', '253', '27.1', '294',
       '515', '612', '1060', '1085', '1274', '130', '1375', '1389', '1669',
       '1697', '1806', '1876', '204', '2052', '605', '738', '821', '1017',
       '1171', '1715', '1767', '1930', '1944', '207', '242', '517', '547',
       '627', '689', '1071', '1559', '1718', '2088', '22.1', '36.1', '381',
       '48.1', '822', '1149', '1229', '1346', '1417', '1645', '1733',
       '390', '476', '676', '891', '930', '117', '1231', '1310', '1536',
       '2261', '309', '34.1', '35.3', '4.2', '409', '47', '559', '58',
       '707', '1196', '1208', '1953', '1971', '2190', '362', '389', '40.4',
       '41.1', '1125', '1139', '1167', '118', '1245', '1263', '1302',
       '1510', '1658', '1674', '183', '1839', '1967', '585', '640', '7',
       '788', '828', '1275', '140', '15', '1612', '1678', '1729', '251',
       '321', '594', '600', '611', '99', '100

In [7]:
import torch.nn.functional as F
F.tanh(model.combined_feature_vector(features))

Variable containing:
-1.2799e-01 -3.0379e-02 -4.7480e-01  ...   1.2660e-01  6.1756e-02 -3.1299e-01
-2.5744e-01 -2.1584e-02 -5.4953e-01  ...   1.9152e-01 -1.8904e-02 -4.0735e-01
-1.5847e-01  1.1012e-01 -4.7973e-01  ...   1.6229e-01  2.5392e-01 -4.0335e-01
                ...                   ⋱                   ...                
-2.9572e-01 -1.9766e-02 -5.5622e-01  ...   2.1145e-01 -3.4282e-02 -4.2156e-01
-2.1236e-01 -2.4452e-02 -5.3466e-01  ...   1.2155e-01 -3.6664e-02 -3.5795e-01
-1.3215e-01 -2.3809e-02 -5.6484e-01  ...   1.6987e-01  2.4750e-02 -4.1851e-01
[torch.FloatTensor of size 1000x204]

In [8]:
model.dropout(features)

Variable containing:
-0.3792 -0.0000 -0.9834  ...   0.2067  0.0000 -0.0000
-0.0000 -0.0571 -1.0969  ...   0.0000  0.0000 -0.0000
-0.3504  0.0563 -0.0000  ...   0.0000  0.0191 -0.0000
          ...             ⋱             ...          
-0.3124 -0.0000 -0.9964  ...   0.2556  0.0101 -0.6769
-0.0000 -0.0000 -1.0005  ...   0.3454  0.0000 -0.5915
-0.0000  0.1076 -0.0000  ...   0.0000  0.3264 -0.6988
[torch.FloatTensor of size 1000x204]

In [13]:
features.size()

torch.Size([1000, 204])

In [14]:
model.hidden(features)

Variable containing:
-3.8244e+00  2.6957e+00 -7.2175e-02
-3.9288e+00  2.2850e+00 -1.1860e-01
-4.0627e+00  2.1671e+00  3.0250e-01
                 ⋮                  
-4.5525e+00  2.4905e+00 -8.6415e-02
-3.8286e+00  1.7874e+00  5.5805e-01
-4.4491e+00  2.3969e+00 -2.4469e-01
[torch.FloatTensor of size 1000x3]

In [24]:
# batch.aid.data.numpy()
test = [features.data.cpu().numpy()[5],features.data.cpu().numpy()[7]]

In [25]:
np.array(test).shape

(2, 204)

In [21]:
batch.label

Variable containing:
 1
 1
 1
⋮ 
 1
 2
 1
[torch.LongTensor of size 1000]

In [83]:
if batch.label[1].data.numpy()[0] == 3:
    print("OK")

In [79]:
print(dir(batch.label[1].data.numpy()))

['T', '__abs__', '__add__', '__and__', '__array__', '__array_finalize__', '__array_interface__', '__array_prepare__', '__array_priority__', '__array_struct__', '__array_ufunc__', '__array_wrap__', '__bool__', '__class__', '__complex__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dir__', '__divmod__', '__doc__', '__eq__', '__float__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__iand__', '__ifloordiv__', '__ilshift__', '__imatmul__', '__imod__', '__imul__', '__index__', '__init__', '__init_subclass__', '__int__', '__invert__', '__ior__', '__ipow__', '__irshift__', '__isub__', '__iter__', '__itruediv__', '__ixor__', '__le__', '__len__', '__lshift__', '__lt__', '__matmul__', '__mod__', '__mul__', '__ne__', '__neg__', '__new__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rlshift__', '__rmatmul__', '__

In [81]:
batch.label[1].data.numpy()[0]

2

In [19]:
features.data.cpu().numpy()

array([[-0.        ,  0.07533404, -0.        , ...,  0.33050776,
        -0.03647172, -0.        ],
       [-0.50200146, -0.        , -0.        , ...,  0.        ,
        -0.12017281, -0.        ],
       [-0.        , -0.00975599, -0.        , ...,  0.3870208 ,
        -0.        , -0.        ],
       ..., 
       [-0.18306208,  0.16188723, -0.        , ...,  0.16567066,
         0.        , -0.        ],
       [-0.        ,  0.        , -0.        , ...,  0.36261645,
         0.        , -0.        ],
       [-0.38052365, -0.09780853, -0.        , ...,  0.        ,
        -0.        , -0.79321569]], dtype=float32)

In [84]:
batch.label

Variable containing:
 2
 2
 2
⋮ 
 1
 1
 1
[torch.LongTensor of size 1000]

In [99]:
import numpy as np
target_feature = [1,2,3]
features = [[-1,-1,1],[-2,2,-2]]
np.sqrt(np.sum((np.array(features)-target_feature)**2,axis = 1))


array([ 4.12310563,  5.83095189])

In [20]:
print(dir(batch))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'aid', 'answer', 'batch_size', 'dataset', 'ext_feat', 'fromvars', 'label', 'qid', 'question', 'train']


In [110]:
print(type(train))

<class 'SST1.SubjDataset'>


In [111]:
train.fields

{'label': <torchtext.data.field.Field at 0x7f322a56ada0>,
 'text': <torchtext.data.field.Field at 0x7f322a56ad68>}

In [113]:
len(batch.label)

1000

In [114]:
batch.dataset.examples[1]

In [115]:
len(batch.dataset.examples)

7999

In [116]:
print(dir(batch))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'batch_size', 'dataset', 'fromvars', 'label', 'text', 'train']


In [117]:
batch.batch_size

1000

In [120]:
batch.text

Variable containing:
     2    808     21  ...       1      1      1
     4      0      5  ...       1      1      1
  9685      0     46  ...       1      1      1
        ...            ⋱           ...         
  4230   2062      9  ...       1      1      1
    15      9     63  ...       1      1      1
   332     82    908  ...       1      1      1
[torch.LongTensor of size 1000x81]

In [44]:
# batch.dataset.examples[1].text

In [124]:
batch.text

Variable containing:
     2    808     21  ...       1      1      1
     4      0      5  ...       1      1      1
  9685      0     46  ...       1      1      1
        ...            ⋱           ...         
  4230   2062      9  ...       1      1      1
    15      9     63  ...       1      1      1
   332     82    908  ...       1      1      1
[torch.LongTensor of size 1000x81]

In [126]:
print(dir(batch))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'batch_size', 'dataset', 'fromvars', 'label', 'text', 'train']


In [131]:
type(batch.dataset.examples[1])

torchtext.data.example.Example

In [132]:
len(batch.dataset.examples)

7999

In [42]:
# dir(batch)

In [135]:
batch.dataset[5]

In [139]:
batch.text[1].data.numpy()

array([   4,    0,    5, 2274,  699, 1016,    8, 6534,  612,    0,    5,
         18, 3489, 8614,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1])

In [141]:
type(batch.label)

torch.autograd.variable.Variable

In [142]:
batch.label

Variable containing:
 2
 2
 2
⋮ 
 1
 1
 1
[torch.LongTensor of size 1000]

In [143]:
optimizer

NameError: name 'optimizer' is not defined

In [146]:
parameter = filter(lambda p: p.requires_grad, model.parameters())
lr = 1.0
weight_decay = 0
optimizer = torch.optim.Adadelta(parameter, lr=lr, weight_decay=weight_decay)

In [147]:
optimizer

In [148]:
batch.text

Variable containing:
     2    808     21  ...       1      1      1
     4      0      5  ...       1      1      1
  9685      0     46  ...       1      1      1
        ...            ⋱           ...         
  4230   2062      9  ...       1      1      1
    15      9     63  ...       1      1      1
   332     82    908  ...       1      1      1
[torch.LongTensor of size 1000x81]

In [151]:
features = model(batch)


array([[ 0.        ,  0.        ,  0.        , ...,  0.21150009,
         0.        ,  0.19599402],
       [ 0.        ,  0.        ,  0.        , ...,  0.22643675,
         0.2998307 ,  0.15882176],
       [ 0.51109791,  0.43724236,  0.        , ...,  0.        ,
         0.38846686,  0.        ],
       ..., 
       [ 0.28448257,  0.53172576,  0.        , ...,  0.        ,
         0.        ,  0.18477558],
       [ 0.3036153 ,  0.46967146,  0.        , ...,  0.23388135,
         0.39685246,  0.        ],
       [ 0.        ,  0.50386667,  0.        , ...,  0.29729113,
         0.41291639,  0.        ]], dtype=float32)

In [153]:
features.data.cpu().numpy()[2].shape

(300,)

In [155]:
batch.text

Variable containing:
     2    808     21  ...       1      1      1
     4      0      5  ...       1      1      1
  9685      0     46  ...       1      1      1
        ...            ⋱           ...         
  4230   2062      9  ...       1      1      1
    15      9     63  ...       1      1      1
   332     82    908  ...       1      1      1
[torch.LongTensor of size 1000x81]

In [156]:
a = batch.text[2]
b = batch.text[3]

In [158]:
torch.stack([a,b])

Variable containing:

Columns 0 to 12 
 9685     0    46     4  1888  1176  1168     5  2051     0  6811   675    13
    2  9789     0   205  4349    43     8    16   104   153     2    35     9

Columns 13 to 25 
  414   902     0  8597     1     1     1     1     1     1     1     1     1
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 26 to 38 
    1     1     1     1     1     1     1     1     1     1     1     1     1
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 39 to 51 
    1     1     1     1     1     1     1     1     1     1     1     1     1
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 52 to 64 
    1     1     1     1     1     1     1     1     1     1     1     1     1
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 65 to 77 
    1     1     1     1     1     1     1     1     1     1     1     1     1
    1   

In [163]:
a = np.array([[1,2,3]])

In [165]:
a.repeat(1000,axis = 0)

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3],
       ..., 
       [1, 2, 3],
       [1, 2, 3],
       [1, 2, 3]])

In [166]:
batch.text

Variable containing:
     2    808     21  ...       1      1      1
     4      0      5  ...       1      1      1
  9685      0     46  ...       1      1      1
        ...            ⋱           ...         
  4230   2062      9  ...       1      1      1
    15      9     63  ...       1      1      1
   332     82    908  ...       1      1      1
[torch.LongTensor of size 1000x81]

In [179]:
batch.label[4]

Variable containing:
 1
[torch.LongTensor of size 1]

In [181]:
type(TEXT)

torchtext.data.field.Field

In [33]:

index2label = np.array(LABEL.vocab.itos)
index2qid = np.array(QID.vocab.itos)
index2answer = np.array(ANSWER.vocab.itos)

In [34]:
index2answer

array(['<unk>', '<pad>', 'the', ..., 'zvyagintsev', 'zx', 'ø'],
      dtype='<U63')

In [29]:
index2qid

array(['<unk>', '63', '3', ..., '7.1', '9', '94'],
      dtype='<U5')

In [30]:
index2label

array(['<unk>', '0', '1'],
      dtype='<U5')

In [31]:
index2question = np.array(QUESTION.vocab.itos)

In [41]:
index2question
sen = " ".join([index2answer[x] for x in batch.answer[0].data.cpu().numpy()])

" ".join([index2question[x] for x in batch.question[0].data.cpu().numpy()])    

'who discovered radium <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

In [39]:
sen

'a generation later , with the explosions over japan , frederic and irene joliot-curie felt betrayed by their american colleagues . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

In [46]:
batch.answer[0][batch.answer[0] != 1]

Variable containing:
     9
  1522
   275
     3
    18
     2
  5866
    89
   210
     3
 13156
     7
 18836
 22567
  1972
 15727
    21
    56
   107
  2958
     4
[torch.LongTensor of size 21]

In [43]:
# batch.answer[0]

In [14]:
q2neg = {}
q2pos = {}
f = open("data/trecqa.train.tsv")
for l in f:
    ls = l.split("\t")
    qid = int(ls[0])
    aid = int(ls[1])
    if ls[2] == "0":
        if qid not in q2neg:
            q2neg[qid] = [] 
        q2neg[qid].append(aid)
    elif ls[2] == "1":
        if qid not in q2pos:
            q2pos[qid] = [] 
        q2pos[qid].append(aid)

In [13]:
import pandas as pd
nums = []
for key in q2neg:
    nums.append(len(q2neg[key]))
s = pd.Series(nums)
s.describe()

count    1162.000000
mean       40.459552
std        62.189110
min         1.000000
25%         8.000000
50%        21.000000
75%        46.000000
max       748.000000
dtype: float64

In [15]:
nums = []
for key in q2pos:
    nums.append(len(q2pos[key]))
s = pd.Series(nums)
s.describe()

count    1229.000000
mean        5.209927
std         6.809162
min         1.000000
25%         1.000000
50%         3.000000
75%         6.000000
max        51.000000
dtype: float64

In [23]:
batch.ext_feat.size()

torch.Size([1000, 4])

In [26]:
q = batch.question[1]
q[q!=1]

Variable containing:
   9
 747
 362
 616
   7
  93
 280
  22
 207
   2
[torch.LongTensor of size 10]

In [31]:
Variable(torch.ones(2))

Variable containing:
 1
 1
[torch.FloatTensor of size 2]

In [34]:
torch.max(torch.zeros(1),torch.ones(1))


 1
[torch.FloatTensor of size 1]

In [42]:
a = torch.zeros(2)
type(a)
a.shape

torch.Size([2])

In [40]:
torch.mm(a,a)


 0  0
 0  0
[torch.FloatTensor of size 2x2]

torch.Size([2, 2])

In [43]:
type(a)

torch.FloatTensor

In [45]:
a.long()


 0
 0
[torch.LongTensor of size 2]

In [46]:
batch.batch_size

1000

In [48]:
from torch.autograd import Function
from torch.autograd import Variable

class PairwiseLossCriterion(Function):
    """docstring for PairwiseLossCriterion"""
    def __init__(self):
        super(PairwiseLossCriterion, self).__init__()
        self.input = None

    def forward(self, input, weight = None, bias = None):
        self.input = input
        return torch.max(torch.zeros(1), 1 - (input[0] - input[1]))/2

    def backward(self, grad_output):
        diff = 1 - (self.input[0] - self.input[1])
        # print("diff:",diff) #, "grad_output:",grad_output, "type of grad_output:", type(grad_output)
        grad_output = torch.zeros(2)
        if diff.numpy()[0] > 0:
            grad_output[0] = -0.5
            grad_output[1] = 0.5
        else:
            grad_output[0] = 0
            grad_output[1] = 0
        return grad_output.view(2, 1)

In [51]:
from torch.autograd import gradcheck

# gradchek takes a tuple of tensor as input, check if your gradient
# evaluated with these tensors are close enough to numerical
# approximations and returns True if they all verify this condition.
input = (Variable(torch.randn(20,20).float(), requires_grad=True), Variable(torch.randn(20,20).float(), requires_grad=True),)
test = gradcheck(PairwiseLossCriterion.backward, input, eps=1e-6, atol=1e-4)
print(test)

AttributeError: 'Variable' object has no attribute 'input'

In [54]:
# Inherit from Function
class LinearFunction(Function):

    # Note that both forward and backward are @staticmethods
    @staticmethod
    # bias is an optional argument
    def forward(ctx, input, weight, bias=None):
        ctx.save_for_backward(input, weight, bias)
        output = input.mm(weight.t())
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    # This function has only a single output, so it gets only one gradient
    @staticmethod
    def backward(ctx, grad_output):
        # This is a pattern that is very convenient - at the top of backward
        # unpack saved_tensors and initialize all gradients w.r.t. inputs to
        # None. Thanks to the fact that additional trailing Nones are
        # ignored, the return statement is simple even when the function has
        # optional inputs.
        input, weight, bias = ctx.saved_variables
        grad_input = grad_weight = grad_bias = None

        # These needs_input_grad checks are optional and there only to
        # improve efficiency. If you want to make your code simpler, you can
        # skip them. Returning gradients for inputs that don't require it is
        # not an error.
        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0).squeeze(0)

        return grad_input, grad_weight, grad_bias

from torch.autograd import gradcheck

# gradchek takes a tuple of tensor as input, check if your gradient
# evaluated with these tensors are close enough to numerical
# approximations and returns True if they all verify this condition.
input = (Variable(torch.randn(20,20).double(), requires_grad=True), Variable(torch.randn(30,20).double(), requires_grad=True),)
test = gradcheck(LinearFunction.apply, input, eps=1e-6, atol=1e-4)
print(test)

False


In [8]:
a = torch.ones(1)

In [9]:
type(a)

torch.FloatTensor

In [10]:
a = {1:2}
a.keys

<function dict.keys>

In [21]:
index2qid = np.array(QID.vocab.itos)
index2qid[1360]


'1734'

In [17]:
index2question = np.array(QUESTION.vocab.itos)
index2question

array(['<unk>', '<pad>', '?', ..., 'waltz', 'yaroslavl', 'zero'],
      dtype='<U18')

In [19]:
index2answer = np.array(ANSWER.vocab.itos)
index2answer[30299]

'orpheus'

In [20]:
index2aid = np.array(AID.vocab.itos)
index2aid[30299]

'36635'

In [29]:
a = [1,2,3,4]
a = np.array(a)

In [30]:
a[np.array([True,True,True,False])]

TypeError: only integer scalar arrays can be converted to a scalar index

In [40]:
pos_feature = np.array([3,4])
neg_feature = np.array([1,-1])

pos_feature_norm = pos_feature / np.sqrt(sum(pos_feature ** 2))
neg_feature_norm = neg_feature / np.sqrt(sum(neg_feature ** 2))

dis = 1 - neg_feature_norm.dot(pos_feature_norm)

In [41]:
dis

1.1414213562373097

In [46]:
d = {1:3,-1:4}
sorted(d.items(), key=lambda t: t[0])

[(-1, 4), (1, 3)]